In [1]:
import numpy as np
import pandas as pd
import copy as cp
from sklearn import datasets
from tidyframe import nest, unnest, apply_window

# Test 1

In [2]:
iris = datasets.load_iris()
df = pd.DataFrame(iris['data'], columns=iris.feature_names)
df['target'] = iris.target
df['target2'] = list(map(lambda x: str(x +1),df.target))
df = df.replace({'target':{key:value for key, value in zip([0,1,2], iris['target_names'])}})
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,target2
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,1
3,4.6,3.1,1.5,0.2,setosa,1
4,5.0,3.6,1.4,0.2,setosa,1


In [3]:
columns = df.columns[:4].tolist()
df_nest= nest(df, columns, key='data_nest', copy=True).head()
df_nest['shape'] = df_nest['data_nest'].apply(lambda x: x.shape)
df_nest

,target,target2,data_nest,shape
0,setosa,1,sepal length (cm) sepal width (cm) petal...,"(50, 4)"
1,versicolor,2,sepal length (cm) sepal width (cm) petal...,"(50, 4)"
2,virginica,3,sepal length (cm) sepal width (cm) peta...,"(50, 4)"


In [4]:
nest(df.groupby(['target', 'target2']), key='data_nest', copy=True).head()

,target,target2,data_nest
0,setosa,1,petal length (cm) petal width (cm) sepal...
1,versicolor,2,petal length (cm) petal width (cm) sepal...
2,virginica,3,petal length (cm) petal width (cm) sepa...


In [5]:
nest(df, df.columns[:4], key='data_nest', copy=True).head()

,target,target2,data_nest
0,setosa,1,sepal length (cm) sepal width (cm) petal...
1,versicolor,2,sepal length (cm) sepal width (cm) petal...
2,virginica,3,sepal length (cm) sepal width (cm) peta...


In [6]:
nest(df, columns_minus=df.columns[4:6].tolist(), key='data_nest', copy=True).head()

,target,target2,data_nest
0,setosa,1,petal length (cm) petal width (cm) sepal...
1,versicolor,2,petal length (cm) petal width (cm) sepal...
2,virginica,3,petal length (cm) petal width (cm) sepa...


In [7]:
nest(df, columns_minus=df.columns[4:6], key='data_nest', copy=True).head()

,target,target2,data_nest
0,setosa,1,petal length (cm) petal width (cm) sepal...
1,versicolor,2,petal length (cm) petal width (cm) sepal...
2,virginica,3,petal length (cm) petal width (cm) sepa...


In [8]:
nest(df, columns_between=['sepal length (cm)', 'petal width (cm)'])

,target,target2,data
0,setosa,1,sepal length (cm) sepal width (cm) petal...
1,versicolor,2,sepal length (cm) sepal width (cm) petal...
2,virginica,3,sepal length (cm) sepal width (cm) peta...


# Test 2 

In [9]:
unnest(df_nest)[df.columns].head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,target2
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,1
3,4.6,3.1,1.5,0.2,setosa,1
4,5.0,3.6,1.4,0.2,setosa,1


# Test3 nest Series or list 

In [10]:
df_string = pd.DataFrame()
df_string['a'] = list('11223')
df_string['b'] = list('22334')
df_string['c'] = ['a', 'bb', 'ccc','dddd','eeeee']
df_string['d'] = df_string['c'].map(lambda x: list(x))

In [11]:
unnest(df_string)

,a,b,c,d
0,1,2,a,a
1,1,2,bb,b
2,1,2,bb,b
3,2,3,ccc,c
4,2,3,ccc,c
5,2,3,ccc,c
6,2,3,dddd,d
7,2,3,dddd,d
8,2,3,dddd,d
9,2,3,dddd,d


In [12]:
# Test4: unnest string spilit

In [13]:
df_comma = pd.DataFrame({'a':list('12'), "b":['A,B,C', 'D,E,F,G']})
df_comma['b_split'] = df_comma['b'].apply(lambda x: x.split(','))
unnest(df_comma)

,a,b,b_split
0,1,"A,B,C",A
1,1,"A,B,C",B
2,1,"A,B,C",C
3,2,"D,E,F,G",D
4,2,"D,E,F,G",E
5,2,"D,E,F,G",F
6,2,"D,E,F,G",G


# partition function

# Test field apply

In [14]:
apply_window(df, np.mean, partition=['target'], columns=df.columns[1])[:10]

0    3.418
1    3.418
2    3.418
3    3.418
4    3.418
5    3.418
6    3.418
7    3.418
8    3.418
9    3.418
Name: sepal width (cm), dtype: float64

In [15]:
apply_window(df, np.mean, partition=['target'], columns=df.columns[:3]).head()

,sepal length (cm)_mean,sepal width (cm)_mean,petal length (cm)_mean
0,5.006,3.418,1.464
1,5.006,3.418,1.464
2,5.006,3.418,1.464
3,5.006,3.418,1.464
4,5.006,3.418,1.464


In [16]:
field_apply = {k:v for (k, v) in (zip(df.columns[:2], [np.max, np.mean]))}
field_apply['sepal length (cm)'] = [np.min, np.max]
field_apply

{'sepal length (cm)': [<function numpy.core.fromnumeric.amin>,
  <function numpy.core.fromnumeric.amax>],
 'sepal width (cm)': <function numpy.core.fromnumeric.mean>}

In [17]:
apply_window(df, field_apply, partition='target').head()

,sepal length (cm)_amin,sepal length (cm)_amax,sepal width (cm)_mean
0,4.3,5.8,3.418
1,4.3,5.8,3.418
2,4.3,5.8,3.418
3,4.3,5.8,3.418
4,4.3,5.8,3.418


In [18]:
apply_window(df.groupby('target'), field_apply).head()

,sepal length (cm)_amin,sepal length (cm)_amax,sepal width (cm)_mean
0,4.3,5.8,3.418
1,4.3,5.8,3.418
2,4.3,5.8,3.418
3,4.3,5.8,3.418
4,4.3,5.8,3.418


In [19]:
df_g = df.groupby('target')

In [20]:
apply_window(df_g, [np.mean, np.max], columns=['sepal length (cm)', 'sepal length (cm)']).head()

,sepal length (cm)_mean,sepal length (cm)_mean,sepal length (cm)_amax,sepal length (cm)_amax
0,5.006,5.006,5.8,5.8
1,5.006,5.006,5.8,5.8
2,5.006,5.006,5.8,5.8
3,5.006,5.006,5.8,5.8
4,5.006,5.006,5.8,5.8


In [21]:
apply_window(df_g, [np.mean, np.max], columns='sepal length (cm)').head()

0    5.006
1    5.006
2    5.006
3    5.006
4    5.006
Name: sepal length (cm), dtype: float64